In [1]:
import cv2
import numpy as np

# --- Cargar imágenes de referencia ---
img_alto = cv2.imread('alto.jpg', cv2.IMREAD_GRAYSCALE)
img_ceda = cv2.imread('resbalar.jpg', cv2.IMREAD_GRAYSCALE)

if img_alto is None or img_ceda is None:
    print("No se encontró una o más imágenes de referencia.")
    exit()

# --- Detector y Matcher ---
sift = cv2.SIFT_create()
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

# --- Detectar keypoints y descriptores de ambas referencias ---
kp_alto, des_alto = sift.detectAndCompute(img_alto, None)
kp_ceda, des_ceda = sift.detectAndCompute(img_ceda, None)

# --- Parámetros para detección por tamaño ---
pix_cerca = 250
pix_medio = 150

# --- Captura de video ---
cap = cv2.VideoCapture(0)

def detectar_y_manejar(frame, img_ref, kp_ref, des_ref, nombre_senal, color):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kp_frame, des_frame = sift.detectAndCompute(gray, None)
    if des_frame is None or len(kp_frame) == 0:
        return frame, False

    matches = bf.knnMatch(des_ref, des_frame, k=2)
    good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]

    if len(good_matches) >= 10:
        pts_ref = np.float32([kp_ref[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        pts_frame = np.float32([kp_frame[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(pts_ref, pts_frame, cv2.RANSAC, 5.0)

        if H is not None:
            h, w = img_ref.shape
            pts_corners = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
            pts_transformed = cv2.perspectiveTransform(pts_corners, H)
            frame = cv2.polylines(frame, [np.int32(pts_transformed)], True, color, 3)

            w_obj = np.linalg.norm(pts_transformed[0] - pts_transformed[1])
            cv2.putText(frame, f"{nombre_senal} - Ancho: {int(w_obj)}px", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

            if w_obj > pix_cerca:
                estado = "MUY CERCA - DETENERSE"
            elif w_obj > pix_medio:
                estado = "ACERCANDOSE"
            else:
                estado = "LEJOS"

            cv2.putText(frame, estado, (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            return frame, True

    return frame, False

# --- Loop principal ---
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame, detectado_alto = detectar_y_manejar(frame, img_alto, kp_alto, des_alto, "ALTO", (0, 255, 0))
    frame, detectado_ceda = detectar_y_manejar(frame, img_ceda, kp_ceda, des_ceda, "CEDA", (255, 0, 0))

    if not detectado_alto and not detectado_ceda:
        cv2.putText(frame, "Ninguna senal detectada", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Detección de señales", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
